In [20]:
import random

%run Operation.ipynb
%run SectionAnalyser.ipynb
%run Account.ipynb
%run DataSource.ipynb


class OperationRegression:
    
    def __init__(self, symbols,start_date, end_date, short, mid):
        self.symbols = symbols
        self.start_date = start_date
        self.end_date = end_date
        self.short = short
        self.mid = mid
        
        self.sectionAnalyser = SectionAnalyser(symbols,short=30, mid=60)
        
        self.resonate_sections = self.sectionAnalyser.get_cross_resonate_on_week_and_day()
        
        
    def find_today_entry_stocks(self, cur_date):
        part = self.resonate_sections[self.resonate_sections['d_s_date']==cur_date]
        
        if(part.shape[0]==0):
            return None
        else:
            return part['symbol'].values
        
    def random_pick_n(self, stocks, n):
        return random.sample(set(stocks), n)
    
    def get_open_price(self, cur_date, d_k):
        return d_k[d_k['date']==cur_date]['close'].values[0]
        
        
    def run(self):
        account = Account(100000, self.start_date, self.end_date,self.short, self.mid)
        all_dates = get_all_dates_between(self.start_date, self.end_date)
        
        operation_df = pd.DataFrame(columns=['date','profit_rate','cash'])
        
        for cur_date in all_dates:
            stocks = self.find_today_entry_stocks(cur_date)
            
            if(stocks is not None):
                symbol = self.random_pick_n(stocks, 1)
                
                operate_symbol = symbol[0]
                
                day_dataSource = DataSource(operate_symbol,level_day)
                d_k = day_dataSource.get_data()
                
                operation = Operation(account, operate_symbol,self.short, self.mid)
                operation.open_opsition(cur_date, self.get_open_price(cur_date, d_k))
                operation.watching_for_sell(cur_date)
                
                temp_section = operation.get_operations()
                if(len(temp_section)>0):
                    operation_df = pd.concat([temp_section,operation_df], ignore_index=True, sort=False)
                
        return operation_df
    